## Impact of Broker failure

#### See how the topics are currently set up. Leader, Replicas, ISRs

In [ ]:
! kafka-topics.sh --bootstrap-server=kafka1:9092 \
                  --describe

#### Kill Broker 2

This needs to be run in your host environment that is running the Docker containers
```
docker kill $(docker ps | grep kafka2 | awk '{print $1}')
```

#### See how the topics are changed. Leader, Replicas, ISRs

In [ ]:
! kafka-topics.sh --bootstrap-server=kafka1:9092 \
                  --describe

Send a few records

In [ ]:
! kafka-producer-perf-test.sh --producer-props bootstrap.servers=kafka1:9092 \
                                               acks=all \
                              --topic test-topic \
                              --num-records   10 \
                              --record-size 2048 \
                              --throughput    -1

However, our producer cannot produce since with ```acks=all``` we no longer have enough ISRs. Let's loosen it a bit.

In [ ]:
! kafka-configs.sh --bootstrap-server kafka1:9092 \
                   --entity-type topics \
                   --entity-name test-topic \
                   --alter \
                   --add-config min.insync.replicas=2

! kafka-topics.sh --bootstrap-server=kafka1:9092 \
                  --describe

In [ ]:
! kafka-producer-perf-test.sh --producer-props bootstrap.servers=kafka1:9092 \
                                               acks=all \
                              --topic test-topic \
                              --num-records   10 \
                              --record-size 2048 \
                              --throughput    -1

#### Bring broker 2 back

This needs to be run in your host environment that is running the Docker containers

```docker compose up -d kafka2```

In [ ]:
! kafka-topics.sh --bootstrap-server=kafka1:9092 \
                  --describe

Kafka's ```auto.leader.rebalance.enable``` is set to true, and ```leader.imbalance.check.interval.seconds``` is set to 5 minutes, the leaders balance out in a bit.

In [ ]:
! kafka-topics.sh --bootstrap-server=kafka3:9092 \
                  --describe